# SKTime
## Imports

In [1]:
import pandas as pd
from sktime.split import temporal_train_test_split

## Data 

In [2]:
df = pd.read_csv('datawarehouse/Merged.csv')
df.head(3)

,AID_YEAR,GROUP_ID,ACADEMIC_LEVEL_TERM_START,ACADEMIC_PLAN,ACADEMIC_PROGRAM_DESC,FIN_AID_FED_RES,UVA_ACCESS,REPORT_CODE,Need based,INST_NEED,OFFER_BALANCE,FUNDED_PARTY,TOTAL_PARTY
0,2015,6,Level Four,AERO-2MJ,Engineering,N,Federal - Grants,FE9,Y,286760.0,25932.0,1.0,1.0
1,2016,6,Level Four,AERO-2MJ,Engineering,N,Federal - Grants,FE9,Y,0.0,0.0,0.0,0.0
2,2017,6,Level Four,AERO-2MJ,Engineering,N,Federal - Grants,FE9,Y,0.0,0.0,0.0,0.0


In [3]:
targets = ['OFFER_BALANCE']
date = 'AID_YEAR'

hierarchies = [col for col in df.columns if df[col].dtype == 'object']
print(f'Hierarchies: {hierarchies}\nTarget {targets} Timepoint {date}.')
print(df[date].min(), df[date].max())

Hierarchies: ['ACADEMIC_LEVEL_TERM_START', 'ACADEMIC_PLAN', 'ACADEMIC_PROGRAM_DESC', 'FIN_AID_FED_RES', 'UVA_ACCESS', 'REPORT_CODE', 'Need based']
Target ['OFFER_BALANCE'] Timepoint AID_YEAR.
2015 2024


In [4]:
min_year = df[date].min()
df[date] -= min_year
df = df[hierarchies+[date]+targets]

In [13]:
# take first 10 time series groups
data = df.iloc[:(df[date].max()+1)*10]

In [5]:
df.set_index(hierarchies + [date], inplace=True)
df.head(3)

OFFER_BALANCE
ACADEMIC_LEVEL_TERM_START ACADEMIC_PLAN ACADEMIC_PROGRAM_DESC FIN_AID_FED_RES UVA_ACCESS       REPORT_CODE Need based AID_YEAR               
Level Four                AERO-2MJ      Engineering           N               Federal - Grants FE9         Y          0               25932.0
                                                                                                                      1                   0.0
                                                                                                                      2                   0.0

In [12]:
# y_train, y_test, X_train, X_test = temporal_train_test_split(data, test_size=)

## Model

In [9]:
from sktime.forecasting.arima import AutoARIMA
forecaster = AutoARIMA(suppress_warnings=True)

In [11]:
data = df.iloc[0:100]

In [7]:
prediction_length = 6
fh = [i for i in range(1, prediction_length+1)]
if forecaster.get_tag("requires-fh-in-fit"):
    forecaster.fit(data, fh=fh)
else: forecaster.fit(data)

KeyboardInterrupt: 

## Forecast

### Predict

In [19]:
# y_pred = forecaster.predict(fh=fh)
# y_pred

INST_NEED  \
ACADEMIC_LEVEL_TERM_START ACADEMIC_PLAN ACADEMIC_PROGRAM_DESC FIN_AID_FED_RES UVA_ACCESS             REPORT_CODE Need based AID_YEAR                 
Level Four                AERO-2MJ      Engineering           N               Departmental Aid       DPT         Y          10         8463.000000   
                                                                                                                            11         7405.125000   
                                                                              Federal - Grants       FE1         Y          10        18031.750000   
                                                                                                                            11        15777.781250   
                                                                                                     FE9         Y          10        42983.825463   
                                                                                                                            11        44006.428945   
                                                                              Federal - Loans        FE10        N          10        18031.750000   
                                                                                                                            11        15777.781250   
                                                                                                     FE4         Y          10        18031.750000   
                                                                                                                            11        15777.781250   
                                                                              Federal Work Study     FE8         Y          10        18031.750000   
                                                                                                                            11        15777.781250   
                                                                              Institutional - Grants IN2         Y          10        18031.750000   
                                                                                                                            11        15777.781250   
                                                                              Institutional - Loans  IN3         Y          10        18031.750000   
                                                                                                                            11        15777.781250   
                                                                              Outside - Grants       OT3         N          10        14641.947301   
                                                                                                                            11        10810.761680   
                                                              V               Departmental Aid       DPT         Y          10        14730.183344   
                                                                                                                            11        11117.971962   

                                                                                                                                      OFFER_BALANCE  \
ACADEMIC_LEVEL_TERM_START ACADEMIC_PLAN ACADEMIC_PROGRAM_DESC FIN_AID_FED_RES UVA_ACCESS             REPORT_CODE Need based AID_YEAR                  
Level Four                AERO-2MJ      Engineering           N               Departmental Aid       DPT         Y          10           216.717873   
                                                                                                                            11           192.037149   
                                                                              Federal - Grants       FE1         Y          10           777.223213   
                                                                                 

### Coverage

In [20]:
y_coverage = forecaster.predict_interval(fh=fh, coverage=0.9)
y_coverage

INST_NEED  \
                                                                                                                                                0.9   
                                                                                                                                              lower   
ACADEMIC_LEVEL_TERM_START ACADEMIC_PLAN ACADEMIC_PROGRAM_DESC FIN_AID_FED_RES UVA_ACCESS             REPORT_CODE Need based AID_YEAR                  
Level Four                AERO-2MJ      Engineering           N               Departmental Aid       DPT         Y          10        -26260.568958   
                                                                                                                            11        -27588.670360   
                                                                              Federal - Grants       FE1         Y          10        -55952.264481   
                                                                                                                            11        -58781.993000   
                                                                                                     FE9         Y          10       -105600.520626   
                                                                                                                            11       -104619.959401   
                                                                              Federal - Loans        FE10        N          10        -55952.264481   
                                                                                                                            11        -58781.993000   
                                                                                                     FE4         Y          10        -55952.264481   
                                                                                                                            11        -58781.993000   
                                                                              Federal Work Study     FE8         Y          10        -55952.264481   
                                                                                                                            11        -58781.993000   
                                                                              Institutional - Grants IN2         Y          10        -55952.264481   
                                                                                                                            11        -58781.993000   
                                                                              Institutional - Loans  IN3         Y          10        -55952.264481   
                                                                                                                            11        -58781.993000   
                                                                              Outside - Grants       OT3         N          10        -41242.754448   
                                                                                                                            11        -46955.344050   
                                                              V               Departmental Aid       DPT         Y          10        -23362.158825   
                                                                                                                            11        -28102.998876   

                                                                                                                                                     \
                                                                                                                                                      
                                                                                                                                              upper   
ACADEMIC_LEVEL_TERM_START ACADEMIC_PLAN ACADEMIC_PROGRAM_DES

### Quantiles

In [29]:
y_quantiles = forecaster.predict_quantiles(fh=fh)

In [30]:
y_quantiles

INST_NEED  \
                                                                                                                                          0.05   
ACADEMIC_LEVEL_TERM_START ACADEMIC_PLAN ACADEMIC_PROGRAM_DESC FIN_AID_FED_RES UVA_ACCESS         REPORT_CODE Need based AID_YEAR                 
Level Two                 UNDBS-ENU     Engineering           V               Endowment - Grants EF1         Y          10       -17103.246579   
                                                                                                                        11       -20712.305465   
                                                                                                 EF3         N          10       -25815.286747   
                                                                                                                        11       -27591.257170   
                                                                              Federal - Grants   FE1         Y          10       -39668.203709   
...                                                                                                                                        ...   
                          YOUTH-BSED    Education             V               State - Grants     ST10        N          11        -8021.291174   
                                                                                                 ST4         Y          10       -22100.424531   
                                                                                                                        11       -21348.444598   
                                                                                                 ST9         Y          10       -14300.110131   
                                                                                                                        11       -15023.323568   

                                                                                                                                                \
                                                                                                                                          0.95   
ACADEMIC_LEVEL_TERM_START ACADEMIC_PLAN ACADEMIC_PROGRAM_DESC FIN_AID_FED_RES UVA_ACCESS         REPORT_CODE Need based AID_YEAR                 
Level Two                 UNDBS-ENU     Engineering           V               Endowment - Grants EF1         Y          10        48277.070644   
                                                                                                                        11        45873.140483   
                                                                                                 EF3         N          10        45671.417788   
                                                                                                                        11        44617.706225   
                                                                              Federal - Grants   FE1         Y          10        78376.611356   
...                                                                                                                                        ...   
                          YOUTH-BSED    Education             V               State - Grants     ST10        N          11        12327.570254   
                                                                                                 ST4         Y          10        35252.223159   
                                                                                                                        11        37999.213927   
                                                                                                 ST9         Y          10        23517.110131   
                                                                                                                        11        23088.198568   

                                           

## Update